#  Train a GPT-2 Text-Generating Model w/ GPU For Free 

by [Max Woolf](http://minimaxir.com)

*Last updated: November 10th, 2019*

Retrain an advanced text generating neural network on any text dataset **for free on a GPU using Collaboratory** using `gpt-2-simple`!

For more about `gpt-2-simple`, you can visit [this GitHub repository](https://github.com/minimaxir/gpt-2-simple). You can also read my [blog post](https://minimaxir.com/2019/09/howto-gpt2/) for more information how to use this notebook!


To get started:

1. Copy this notebook to your Google Drive to keep it and save your changes. (File -> Save a Copy in Drive)
2. Make sure you're running the notebook in Google Chrome.
3. Run the cells below:


In [ ]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

TensorFlow 1.x selected.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



## Downloading GPT-2

If you're retraining a model on new text, you need to download the GPT-2 model first. 

There are three released sizes of GPT-2:

* `124M` (default): the "small" model, 500MB on disk.
* `355M`: the "medium" model, 1.5GB on disk.
* `774M`: the "large" model, cannot currently be finetuned with Colaboratory but can be used to generate text from the pretrained model (see later in Notebook)
* `1558M`: the "extra large", true model. Will not work if a K80 GPU is attached to the notebook. (like `774M`, it cannot be finetuned).

Larger models have more knowledge, but take longer to finetune and longer to generate text. You can specify which base model to use by changing `model_name` in the cells below.

The next cell downloads it from Google Cloud Storage and saves it in the Colaboratory VM at `/models/<model_name>`.

This model isn't permanently saved in the Colaboratory VM; you'll have to redownload it if you want to retrain it at a later time.

In [ ]:
gpt2.download_gpt2(model_name="355M")

Fetching checkpoint: 1.05Mit [00:00, 319Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 131Mit/s]                                                    
Fetching hparams.json: 1.05Mit [00:00, 774Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 1.42Git [00:05, 265Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 285Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 113Mit/s]                                                 
Fetching vocab.bpe: 1.05Mit [00:00, 152Mit/s]                                                       


## Mounting Google Drive

The best way to get input text to-be-trained into the Colaboratory VM, and to get the trained model *out* of Colaboratory, is to route it through Google Drive *first*.

Running this cell (which will only work in Colaboratory) will mount your personal Google Drive in the VM, which later cells can use to get data in/out. (it will ask for an auth code; that auth is not saved anywhere)

In [ ]:
gpt2.mount_gdrive()

Mounted at /content/drive


## Generating a text-file for the training-process



**A simple pasting together for all papers**

In [ ]:
import os
dir = 'drive/MyDrive/NLP_scientific-text-generation/02_fulltexts-climate-change'

with open ("fulltexts.txt", "w") as out_file:
  for f_name in os.listdir(dir):
    path = os.path.join(dir, f_name)
    with open(path, 'r') as f:
      f_text = ''.join(f.readlines())
      f_text += "\n"
    out_file.write("%s\n" % f_text)

**Each paper in a separated line**

In [ ]:
import re

dir = 'drive/MyDrive/NLP_scientific-text-generation/02_fulltexts-climate-change'
papers = []

for f_name in os.listdir(dir):
  path = os.path.join(dir, f_name)
  with open(path, "r") as f:
    f_text = ''.join(f.readlines()) # Fulltext as a single string
    f_text = re.sub(r"\n[^\n]", "", f_text) # Remove Carriage Return due to Scanning (i.e. only if no carriage return follows immediatly)
    papers.append(f_text)

with open('fulltext_papers_linewise.txt', 'w') as f:
    for item in papers:
        f.write("%s\n" % item)

**remove short lines in order to get rid of tables etc...**

In [ ]:
import re
import os

dir = 'drive/MyDrive/NLP_scientific-text-generation/02_fulltexts-climate-change'


with open("full_texts_merged.txt", "w") as out_file:
  for f_name in os.listdir(dir):
    path = os.path.join(dir, f_name)
    with open(path, "r") as f:
      lines = []
      for line in f:
        if len(line) > 20 or line == "\n": # Remove short lines that are likely result of tables
          lines.append(line)
      f_text = "".join(lines) # Fulltext as a single string
      # Remove Carriage Return due to Scanning (i.e. only if no carriage return follows immediatly)
      f_text = re.sub(r"\n(?!\n)", " ", f_text)
      # Strip leading whitespaces left over from previous step, remove empty lines
      f_text = re.sub(r"(\s?\n\s?)+", "\n", f_text)
      # Strip lines shorter than 50 chars, only leaving full paragraphs of text
      f_text = re.sub(r"\n(.{0,50})(?=\n)", "", f_text)
      # Remove leading and trailing whitespace
      f_text = f_text.strip()
      out_file.write(f_text)

**Specify the file you want to use for training and name your model**

In [ ]:
#The name of the text-file for fine-tuning
file_name = "fulltexts.txt"

## Here I name my model, which will be fine-tuned and saved in my gdrive later
my_model_name = "train_2000_fulltext_new"

## Finetune GPT-2

The next cell will start the actual finetuning of GPT-2. It creates a persistent TensorFlow session which stores the training config, then runs the training for the specified number of `steps`. (to have the finetuning run indefinitely, set `steps = -1`)

The model checkpoints will be saved in `/checkpoint/run1` by default. The checkpoints are saved every 500 steps (can be changed) and when the cell is stopped.

The training might time out after 4ish hours; make sure you end training and save the results so you don't lose them!

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

Other optional-but-helpful parameters for `gpt2.finetune`:


*  **`restore_from`**: Set to `fresh` to start training from the base GPT-2, or set to `latest` to restart training from an existing checkpoint.
* **`sample_every`**: Number of steps to print example output
* **`print_every`**: Number of steps to print training progress.
* **`learning_rate`**:  Learning rate for the training. (default `1e-4`, can lower to `1e-5` if you have <1MB input data)
*  **`run_name`**: subfolder within `checkpoint` to save the model. This is useful if you want to work with multiple models (will also need to specify  `run_name` when loading the model)
* **`overwrite`**: Set to `True` if you want to continue finetuning an existing model (w/ `restore_from='latest'`) without creating duplicate copies. 

In [ ]:
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name='355M',
              steps=2000,
              restore_from='fresh',
              run_name=my_model_name,
              print_every=10,
              sample_every=200,
              save_every=200
              )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Please use tensorflow.python.ops.op_selector.get_backward_walk_ops.
Loading checkpoint models/355M/model.ckpt
INFO:tensorflow:Restoring parameters from models/355M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:07<00:00,  7.92s/it]


dataset has 1623470 tokens
Training...
[10 | 23.36] loss=2.73 avg=2.73
[20 | 38.34] loss=3.27 avg=3.00
[30 | 53.58] loss=2.73 avg=2.91
[40 | 69.15] loss=3.32 avg=3.01
[50 | 85.04] loss=3.31 avg=3.07
[60 | 101.34] loss=3.13 avg=3.08
[70 | 118.12] loss=2.83 avg=3.05
[80 | 134.64] loss=3.04 avg=3.04
[90 | 150.87] loss=2.87 avg=3.02
[100 | 167.08] loss=3.20 avg=3.04
[110 | 183.40] loss=3.12 avg=3.05
[120 | 199.84] loss=2.94 avg=3.04
[130 | 216.35] loss=2.57 avg=3.00
[140 | 232.81] loss=2.87 avg=2.99
[150 | 249.25] loss=2.79 avg=2.98
[160 | 265.66] loss=3.26 avg=3.00
[170 | 282.03] loss=2.42 avg=2.96
[180 | 298.40] loss=2.58 avg=2.94
[190 | 314.81] loss=2.77 avg=2.93
[200 | 331.21] loss=2.82 avg=2.92
Saving checkpoint/train_2000_fulltext_new/model-200
======== SAMPLE 1 ========
 the 2014-2015 annual report of the US Agency for International Development [AID] to the International Telecommunication Union [ITU] and to the OECD, the International Telecommunications Union (ITU) and the Organisat

After the model is trained, you can copy the checkpoint folder to your own Google Drive.

If you want to download it to your personal computer, it's strongly recommended you copy it there first, then download from Google Drive. The checkpoint folder is copied as a `.rar` compressed file; you can download it and uncompress it locally.

In [ ]:
gpt2.copy_checkpoint_to_gdrive(run_name=my_model_name)

You're done! Feel free to go to the **Generate Text From The Trained Model** section to generate text based on your retrained model.

## Load a Trained Model Checkpoint

Running the next cell will copy the `.rar` checkpoint file from your Google Drive into the Colaboratory VM.

In [ ]:
gpt2.copy_checkpoint_from_gdrive(run_name=my_model_name)


The next cell will allow you to load the retrained model checkpoint + metadata necessary to generate text.

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

In [ ]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name=my_model_name)

Loading checkpoint checkpoint/train_2000_fulltext_new/model-2000
INFO:tensorflow:Restoring parameters from checkpoint/train_2000_fulltext_new/model-2000


## Generate Text From The Trained Model

After you've trained the model or loaded a retrained model from checkpoint, you can now generate text. `generate` generates a single text from the loaded model.

In [ ]:
gpt2.generate(sess, run_name=my_model_name)

If you're creating an API based on your model and need to pass the generated text elsewhere, you can do `text = gpt2.generate(sess, return_as_list=True)[0]`

You can also pass in a `prefix` to the generate function to force the text to start with a given character sequence and generate text from there (good if you add an indicator when the text starts).

You can also generate multiple texts at a time by specifing `nsamples`. Unique to GPT-2, you can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 20 for `batch_size`).

Other optional-but-helpful parameters for `gpt2.generate` and friends:

*  **`length`**: Number of tokens to generate (default 1023, the maximum)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)
* **`truncate`**: Truncates the input text until a given sequence, excluding that sequence (e.g. if `truncate='<|endoftext|>'`, the returned text will include everything before the first `<|endoftext|>`). It may be useful to combine this with a smaller `length` if the input texts are short.
*  **`include_prefix`**: If using `truncate` and `include_prefix=False`, the specified `prefix` will not be included in the returned text.

For bulk generation, you can generate a large amount of text to a file and sort out the samples locally on your computer. The next cell will generate a generated text file with a unique timestamp.

You can rerun the cells as many times as you want for even more generated texts!

In [ ]:
pre = "[QUESTION]:What is the difference between climate change and global warming? \n[ANSWER]:Global warming refers to the long-term warming of the planet. Climate change encompasses global warming, but refers to the broader range of changes that are happening to our planet, including rising sea levels; shrinking mountain glaciers; accelerating ice melt in Greenland, Antarctica and the Arctic; and shifts in flower/plant blooming times.\n\n"
pre += "[QUESTION]:What is the greenhouse effect? \n[ANSWER]:The greenhouse effect is the way in which heat is trapped close to the surface of the Earth by greenhouse gases.\n\n"
pre+= "[QUESTION]:How much does nuclear power affect world climate change? \n[ANSWER]:"

gpt2.generate(sess,
              run_name=my_model_name,
              length=100,
              temperature=1,
              #truncate = '[QUESTION]',
              prefix=pre,
              nsamples=3,
              batch_size=3
              )

[QUESTION]:What is the difference between climate change and global warming? 
[ANSWER]:Global warming refers to the long-term warming of the planet. Climate change encompasses global warming, but refers to the broader range of changes that are happening to our planet, including rising sea levels; shrinking mountain glaciers; accelerating ice melt in Greenland, Antarctica and the Arctic; and shifts in flower/plant blooming times.

[QUESTION]:What is the greenhouse effect? 
[ANSWER]:The greenhouse effect is the way in which heat is trapped close to the surface of the Earth by greenhouse gases.

[QUESTION]:How much does nuclear power affect world climate change? 
[ANSWER]:Nuclear power releases a lot of greenhouse gases into the atmosphere, as most of the electricity we use comes from fossil-fuel-generated energy industry. Nuclear energy also releases large amounts of heat-trapping gas into the atmosphere, such as methane and carbon dioxide (CO2)
[EXHIBITION]: How much more so if the powe

In [ ]:
pre = "[QUESTION]:What is the difference between climate change and global warming? \n[ANSWER]:Global warming refers to the long-term warming of the planet. Climate change encompasses global warming, but refers to the broader range of changes that are happening to our planet, including rising sea levels; shrinking mountain glaciers; accelerating ice melt in Greenland, Antarctica and the Arctic; and shifts in flower/plant blooming times.\n\n"
pre += "[QUESTION]:What is the greenhouse effect? \n[ANSWER]:The greenhouse effect is the way in which heat is trapped close to the surface of the Earth by greenhouse gases.\n\n"
pre+= "[QUESTION]:What will be the economic effects of climate Change? \n[ANSWER]:"

gpt2.generate(sess,
              run_name=my_model_name,
              length=100,
              temperature=1,
              #truncate = '[QUESTION]',
              prefix=pre,
              nsamples=3,
              batch_size=3
              )

[QUESTION]:What is the difference between climate change and global warming? 
[ANSWER]:Global warming refers to the long-term warming of the planet. Climate change encompasses global warming, but refers to the broader range of changes that are happening to our planet, including rising sea levels; shrinking mountain glaciers; accelerating ice melt in Greenland, Antarctica and the Arctic; and shifts in flower/plant blooming times.

[QUESTION]:What is the greenhouse effect? 
[ANSWER]:The greenhouse effect is the way in which heat is trapped close to the surface of the Earth by greenhouse gases.

[QUESTION]:What will be the economic effects of climate Change? 
[ANSWER]:The effects will be felt in different quantitatively, but most people do not realize that climate change affects every field of human activity, including business. Climate
change induces economic regression and economic stagnation. When given time enough climate change will drive
up energy, food costs, energy, transportation

In [ ]:
pre = "[QUESTION]:What is the difference between climate change and global warming? \n[ANSWER]:Global warming refers to the long-term warming of the planet. Climate change encompasses global warming, but refers to the broader range of changes that are happening to our planet, including rising sea levels; shrinking mountain glaciers; accelerating ice melt in Greenland, Antarctica and the Arctic; and shifts in flower/plant blooming times.\n\n"
pre += "[QUESTION]:What is the greenhouse effect? \n[ANSWER]:The greenhouse effect is the way in which heat is trapped close to the surface of the Earth by greenhouse gases.\n\n"
pre+= "[QUESTION]:What is the advantage and disadvantage of renewable energy? \n[ANSWER]:"

gpt2.generate(sess,
              run_name=my_model_name,
              length=100,
              temperature=1,
              #truncate = '[QUESTION]',
              prefix=pre,
              nsamples=3,
              batch_size=3
              )

[QUESTION]:What is the difference between climate change and global warming? 
[ANSWER]:Global warming refers to the long-term warming of the planet. Climate change encompasses global warming, but refers to the broader range of changes that are happening to our planet, including rising sea levels; shrinking mountain glaciers; accelerating ice melt in Greenland, Antarctica and the Arctic; and shifts in flower/plant blooming times.

[QUESTION]:What is the greenhouse effect? 
[ANSWER]:The greenhouse effect is the way in which heat is trapped close to the surface of the Earth by greenhouse gases.

[QUESTION]:What is the advantage and disadvantage of renewable energy? 
[ANSWER]:There is a huge opportunity for creating jobs and prosperity, as well as providing the environment with what it needs to survive.
This message was distributed in 28 languages.
Globalization is changing how people think and feel. Here are some words and phrases that people used in 288 different languages in 16 countrie

In [ ]:
pre = "[QUESTION]:What is the difference between climate change and global warming? \n[ANSWER]:Global warming refers to the long-term warming of the planet. Climate change encompasses global warming, but refers to the broader range of changes that are happening to our planet, including rising sea levels; shrinking mountain glaciers; accelerating ice melt in Greenland, Antarctica and the Arctic; and shifts in flower/plant blooming times.\n\n"
pre += "[QUESTION]:What is the greenhouse effect? \n[ANSWER]:The greenhouse effect is the way in which heat is trapped close to the surface of the Earth by greenhouse gases.\n\n"
pre+= "[QUESTION]: How much does global warming affect wildlife tourism? \n[ANSWER]:"

gpt2.generate(sess,
              run_name=my_model_name,
              length=100,
              temperature=1,
              #truncate = '[QUESTION]',
              prefix=pre,
              nsamples=3,
              batch_size=3
              )

[QUESTION]:What is the difference between climate change and global warming? 
[ANSWER]:Global warming refers to the long-term warming of the planet. Climate change encompasses global warming, but refers to the broader range of changes that are happening to our planet, including rising sea levels; shrinking mountain glaciers; accelerating ice melt in Greenland, Antarctica and the Arctic; and shifts in flower/plant blooming times.

[QUESTION]:What is the greenhouse effect? 
[ANSWER]:The greenhouse effect is the way in which heat is trapped close to the surface of the Earth by greenhouse gases.

[QUESTION]: How much does global warming affect wildlife tourism? 
[ANSWER]:Climate change can have a negative impact on tourism. Tourism should be able to adapt to some of these changes by choosing areas where the climate is mild enough to allow for tourism; by choosing tourist destinations where there are few or no negative effects on tourism; by choosing places that have low impact on tourism v

In [ ]:
pre = "[QUESTION]:What is the difference between climate change and global warming? \n[ANSWER]:Global warming refers to the long-term warming of the planet. Climate change encompasses global warming, but refers to the broader range of changes that are happening to our planet, including rising sea levels; shrinking mountain glaciers; accelerating ice melt in Greenland, Antarctica and the Arctic; and shifts in flower/plant blooming times.\n\n"
pre += "[QUESTION]:What is the greenhouse effect? \n[ANSWER]:The greenhouse effect is the way in which heat is trapped close to the surface of the Earth by greenhouse gases.\n\n"
pre+= "[QUESTION]: What is the impact from sea level rise on tourism? \n[ANSWER]:"

gpt2.generate(sess,
              run_name=my_model_name,
              length=100,
              temperature=1,
              #truncate = '[QUESTION]',
              prefix=pre,
              nsamples=3,
              batch_size=3
              )

[QUESTION]:What is the difference between climate change and global warming? 
[ANSWER]:Global warming refers to the long-term warming of the planet. Climate change encompasses global warming, but refers to the broader range of changes that are happening to our planet, including rising sea levels; shrinking mountain glaciers; accelerating ice melt in Greenland, Antarctica and the Arctic; and shifts in flower/plant blooming times.

[QUESTION]:What is the greenhouse effect? 
[ANSWER]:The greenhouse effect is the way in which heat is trapped close to the surface of the Earth by greenhouse gases.

[QUESTION]: What is the impact from sea level rise on tourism? 
[ANSWER]: Sea level rise is a phenomenon which, as a result of climate change, raises the risk of erosion of coastal cities. In
this sense, there is increased pressure on the local authorities to reduce the risk of coastal erosion in order to avoid more floods, while the
national framework recommends strengthening the local resilience

In [ ]:
pre = "[QUESTION]:What is the difference between climate change and global warming? \n[ANSWER]:Global warming refers to the long-term warming of the planet. Climate change encompasses global warming, but refers to the broader range of changes that are happening to our planet, including rising sea levels; shrinking mountain glaciers; accelerating ice melt in Greenland, Antarctica and the Arctic; and shifts in flower/plant blooming times.\n\n"
pre += "[QUESTION]:What is the greenhouse effect? \n[ANSWER]:The greenhouse effect is the way in which heat is trapped close to the surface of the Earth by greenhouse gases.\n\n"
pre+= "[QUESTION]: How important is green economy for mitigating global warming? \n[ANSWER]:"

gpt2.generate(sess,
              run_name=my_model_name,
              length=100,
              temperature=1,
              #truncate = '[QUESTION]',
              prefix=pre,
              nsamples=3,
              batch_size=3
              )

[QUESTION]:What is the difference between climate change and global warming? 
[ANSWER]:Global warming refers to the long-term warming of the planet. Climate change encompasses global warming, but refers to the broader range of changes that are happening to our planet, including rising sea levels; shrinking mountain glaciers; accelerating ice melt in Greenland, Antarctica and the Arctic; and shifts in flower/plant blooming times.

[QUESTION]:What is the greenhouse effect? 
[ANSWER]:The greenhouse effect is the way in which heat is trapped close to the surface of the Earth by greenhouse gases.

[QUESTION]: How important is green economy for mitigating global warming? 
[ANSWER]: The green economy refers to areas specifically dedicated to the development of green jobs and the livelihoods of the people.

[QUESTION]: How would a carbon tax help? 
[ANSWER]: Carbon taxes modify the pricing of goods and services on the exchanges. They are therefore needed most urgently in
curbing global warming

In [ ]:
gen_file = 'gpt2_gentext_{:%Y%m%d_%H%M%S}.txt'.format(datetime.utcnow())

gpt2.generate_to_file(sess,
                      destination_path=gen_file,
                      length=500,
                      temperature=0.7,
                      nsamples=100,
                      batch_size=20
                      )

In [ ]:
# may have to run twice to get file to download
files.download(gen_file)

## Generate Text From The Pretrained Model

If you want to generate text from the pretrained model, not a finetuned model, pass `model_name` to `gpt2.load_gpt2()` and `gpt2.generate()`.

This is currently the only way to generate text from the 774M or 1558M models with this notebook.

In [ ]:
model_name = "355M"

In [ ]:
model_name = "355M"

gpt2.download_gpt2(model_name=model_name)

In [ ]:
sess = gpt2.start_tf_sess()

gpt2.load_gpt2(sess, model_name="355M")

Loading pretrained model models/355M/model.ckpt
INFO:tensorflow:Restoring parameters from models/355M/model.ckpt


In [ ]:
pre = "[QUESTION]:What is the difference between climate change and global warming? \n[ANSWER]:Global warming refers to the long-term warming of the planet. Climate change encompasses global warming, but refers to the broader range of changes that are happening to our planet, including rising sea levels; shrinking mountain glaciers; accelerating ice melt in Greenland, Antarctica and the Arctic; and shifts in flower/plant blooming times.\n\n"
pre += "[QUESTION]:What is the greenhouse effect? \n[ANSWER]:The greenhouse effect is the way in which heat is trapped close to the surface of the Earth by greenhouse gases.\n\n"
pre+= "[QUESTION]:How much does nuclear power affect world climate change? \n[ANSWER]:"

gpt2.generate(sess,
              model_name=model_name,
              prefix=pre,
              length=100,
              temperature=0.7,
              top_p=0.9,
              nsamples=3,
              batch_size=3
              )

[QUESTION]:What is the difference between climate change and global warming? 
[ANSWER]:Global warming refers to the long-term warming of the planet. Climate change encompasses global warming, but refers to the broader range of changes that are happening to our planet, including rising sea levels; shrinking mountain glaciers; accelerating ice melt in Greenland, Antarctica and the Arctic; and shifts in flower/plant blooming times.

[QUESTION]:What is the greenhouse effect? 
[ANSWER]:The greenhouse effect is the way in which heat is trapped close to the surface of the Earth by greenhouse gases.

[QUESTION]:How much does nuclear power affect world climate change? 
[ANSWER]:Nuclear power plants are the world's largest source of carbon dioxide, a greenhouse gas. Nuclear power plants are responsible for about a third of all the carbon dioxide emitted by the US, the world's largest emitter. Nuclear power plants are also responsible for nearly all of the carbon dioxide emitted from fossil fuel 

In [ ]:
pre = "[QUESTION]:What is the difference between climate change and global warming? \n[ANSWER]:Global warming refers to the long-term warming of the planet. Climate change encompasses global warming, but refers to the broader range of changes that are happening to our planet, including rising sea levels; shrinking mountain glaciers; accelerating ice melt in Greenland, Antarctica and the Arctic; and shifts in flower/plant blooming times.\n\n"
pre += "[QUESTION]:What is the greenhouse effect? \n[ANSWER]:The greenhouse effect is the way in which heat is trapped close to the surface of the Earth by greenhouse gases.\n\n"
pre+= "[QUESTION]:What will be the economic effects of climate Change? \n[ANSWER]:"

gpt2.generate(sess,
              model_name=model_name,
              prefix=pre,
              length=100,
              temperature=0.7,
              top_p=0.9,
              nsamples=3,
              batch_size=3
              )

[QUESTION]:What is the difference between climate change and global warming? 
[ANSWER]:Global warming refers to the long-term warming of the planet. Climate change encompasses global warming, but refers to the broader range of changes that are happening to our planet, including rising sea levels; shrinking mountain glaciers; accelerating ice melt in Greenland, Antarctica and the Arctic; and shifts in flower/plant blooming times.

[QUESTION]:What is the greenhouse effect? 
[ANSWER]:The greenhouse effect is the way in which heat is trapped close to the surface of the Earth by greenhouse gases.

[QUESTION]:What will be the economic effects of climate Change? 
[ANSWER]:Climate Change is likely to have a negative impact on the economy and adversely affect the quality of life of the people of the world.

[QUESTION]:How much climate change will happen in the next few decades? 

[ANSWER]:The climate change in the next few decades is likely to be much larger than that experienced in the last 10

# Etcetera

If the notebook has errors (e.g. GPU Sync Fail), force-kill the Colaboratory virtual machine and restart it with the command below:

In [ ]:
!kill -9 -1

# LICENSE

MIT License

Copyright (c) 2019 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.